In [24]:
from zurich_move_data_extraction import *
from activity_count_function import *
from utilities import *
from gm_function import *
from individual_analysis_ac_functions import *
from extract_mask_from_video import *
from individual_analysis_fs_functions import *

In [25]:
#PATH 

initial_path = '../CreateStudy'

# Change this value
participant_id = 'HXXX'
dominant_hand = 'Right'

# Join the initial_path and participant_id
participant_path = os.path.join(initial_path, participant_id)
print(participant_path)

../CreateStudy\H001


In [26]:
# Get the data from csv files 
ndh_data, chest_data, dh_data, GT_mask_ndh, GT_mask_dh = get_data(participant_path)
# Ensure the array sizes consistency between masks and IMU data
print (len(GT_mask_ndh)*2, len(ndh_data))

48264 48264


In [27]:
# Get some usefull variables
sampling_freq = 50
acc_ndh = ndh_data[['acc_x', 'acc_y', 'acc_z']]
acc_dh = dh_data[['acc_x', 'acc_y', 'acc_z']]
gyro_ndh = ndh_data[['gyro_x', 'gyro_y', 'gyro_z']]
gyro_dh = dh_data[['gyro_x', 'gyro_y', 'gyro_z']]
mag_ndh = ndh_data[['magneto_x', 'magneto_y', 'magneto_z']]
quat_sensor_ndh = ndh_data[['quat_0', 'quat_1', 'quat_2','quat_3']]
quat_sensor_dh = dh_data[['quat_0', 'quat_1', 'quat_2','quat_3']]
frequency_AC = 1 # Hz 
frequency_GM = 2 # Hz
FPS_mask = 25 # Hz

In [28]:
# Get AC values 
AC_ndh_filename = 'count_brond_ndh.csv'
AC_dh_filename = 'count_brond_dh.csv'

AC_ndh_path = os.path.join(participant_path, AC_ndh_filename)
AC_dh_path = os.path.join(participant_path, AC_dh_filename)

AC_ndh, AC_dh = read_csv_to_numpy(AC_ndh_path, AC_dh_path)

In [29]:
# MASK with WBM replaced by NF #

In [30]:
# Create masks with WBM converted into NF movement

GT_mask_ndh_1Hz = downsample_mask_interpolation(GT_mask_ndh, FPS_mask, frequency_AC)
GT_mask_dh_1Hz = downsample_mask_interpolation(GT_mask_dh, FPS_mask, frequency_AC)
GT_mask_ndh_2Hz = downsample_mask_interpolation(GT_mask_ndh, FPS_mask, frequency_GM)
GT_mask_dh_2Hz = downsample_mask_interpolation(GT_mask_dh, FPS_mask, frequency_GM)

# Replace WBM mask by NF masks @ 25Hz 
GT_mask_ndh_wbm_into_nf = replace_wbm_with_nf(GT_mask_ndh).ravel()
GT_mask_dh_wbm_into_nf = replace_wbm_with_nf(GT_mask_dh).ravel()

# Replace WBM mask by NF masks @ 2Hz 
GT_mask_ndh_2Hz_wbm_into_nf = replace_wbm_with_nf(GT_mask_ndh_2Hz)
GT_mask_dh_2Hz_wbm_into_nf = replace_wbm_with_nf(GT_mask_dh_2Hz)

# Replace WBM mask by NF masks @ 1Hz 
GT_mask_ndh_1Hz_wbm_into_nf = replace_wbm_with_nf(GT_mask_ndh_1Hz)
GT_mask_dh_1Hz_wbm_into_nf = replace_wbm_with_nf(GT_mask_dh_1Hz)

# Trim dataset to ensure same size 
GT_mask_ndh_1Hz_wbm_into_nf, AC_ndh = remove_extra_elements(GT_mask_ndh_1Hz_wbm_into_nf, AC_ndh)
GT_mask_dh_1Hz_wbm_into_nf, AC_dh = remove_extra_elements(GT_mask_dh_1Hz_wbm_into_nf, AC_dh)

# Upsample the masks 
GT_mask_50Hz_ndh = resample_mask(GT_mask_ndh_wbm_into_nf, 25.0, 50.0)
GT_mask_50Hz_dh = resample_mask(GT_mask_dh_wbm_into_nf, 25.0, 50.0)

In [31]:
## GM 

In [32]:
# Get Euler angles 

# Left wrist
roll_mad_ndh, pitch_mad_ndh, yaw_mad_ndh = compute_euler_angles(acc_ndh, gyro_ndh, sampling_freq)
#plot_angles_over_time(pitch_mad_ndh), plot_angles_over_time(yaw_mad_ndh)

# Right wrist 
roll_mad_dh, pitch_mad_dh, yaw_mad_dh = compute_euler_angles(acc_dh, gyro_dh, sampling_freq)
#plot_angles_over_time(pitch_mad_dh), plot_angles_over_time(yaw_mad_dh)

Using IMU Madgwick with acc and gyro only
Using IMU Madgwick with acc and gyro only


In [33]:
# Computation of an optimal FS for the GM algorithm # 

In [ ]:
# Optimal Left wrist 
eval_metrics_fs_opt_ndh, optimal_fs_ndh = optimal_fs_computation(pitch_mad_ndh, yaw_mad_ndh, GT_mask_50Hz_ndh,
                                                               k=5, random_state=42, optimal=True)

# Conventional Left wrist 
eval_metrics_fs_conv_ndh, _ = optimal_fs_computation(pitch_mad_ndh, yaw_mad_ndh, GT_mask_50Hz_ndh,
                                                    k=5, random_state=42, optimal=False)

In [ ]:
# Optimal Right wrist 
eval_metrics_fs_opt_dh, optimal_fs_dh = optimal_fs_computation(pitch_mad_dh, yaw_mad_dh, GT_mask_50Hz_dh,
                                                               k=5, random_state=42, optimal=True)

# Conventional Right wrist 
eval_metrics_fs_conv_dh, _ = optimal_fs_computation(pitch_mad_dh, yaw_mad_dh, GT_mask_50Hz_dh,
                                                    k=5, random_state=42, optimal=False)

In [ ]:
# Optimal Bilateral 
eval_metrics_fs_opt_bil = optimal_fs_computation_bilateral(pitch_mad_ndh, yaw_mad_ndh, GT_mask_50Hz_ndh,
                                     pitch_mad_dh, yaw_mad_dh, GT_mask_50Hz_dh,
                                     optimal_fs_ndh, optimal_fs_dh,k=5, random_state=42, optimal=True)

# Conventional Bilateral 
eval_metrics_fs_conv_bil = optimal_fs_computation_bilateral(pitch_mad_ndh, yaw_mad_ndh, GT_mask_50Hz_ndh,
                                     pitch_mad_dh, yaw_mad_dh, GT_mask_50Hz_dh,
                                     optimal_fs_ndh, optimal_fs_dh,k=5, random_state=42, optimal=False)

In [ ]:
print(optimal_fs_ndh, optimal_fs_dh)

In [ ]:
eval_metrics_fs_conv_bil

In [ ]:
plot_radar_chart(eval_metrics_fs_conv_ndh, eval_metrics_fs_opt_ndh, AC = False)

In [ ]:
plot_radar_chart(eval_metrics_fs_conv_dh, eval_metrics_fs_opt_dh, AC = False)

In [ ]:
plot_radar_chart(eval_metrics_fs_conv_bil, eval_metrics_fs_opt_bil, AC = False)

In [ ]:
# Saving data for this participant 

In [ ]:
# Create a structure to save all the metrics 
metrics_dictionary = create_metrics_dictionary(eval_metrics_fs_conv_ndh, eval_metrics_fs_conv_dh, eval_metrics_fs_conv_bil, eval_metrics_fs_opt_ndh, eval_metrics_fs_conv_dh, eval_metrics_fs_opt_bil)

In [ ]:
metrics_dictionary

In [ ]:
# Save metrics as csv file: evaluation metrics on unseen data 
save_metrics_dictionary_as_csv(metrics_dictionary, participant_path, AC = False)
# Save optimal AC threhshold for this participant 
save_optimal_threshold(participant_path, optimal_fs_ndh, optimal_fs_dh, AC = False)